In [3]:
import os 
from dotenv import load_dotenv


In [4]:
# .envから環境変数を読み込む
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OpenAI APIキーが設定されていません。環境変数 'OPENAI_API_KEY' を設定してください。")


In [10]:
from langchain_openai  import OpenAIEmbeddings

from src.extract_text import extract_text_from_pdf
from src.split_text import split_text
from src.create_embeddings import create_vectorstore

# パス設定
#SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
# パス設定(notebook)
SCRIPT_DIR = os.path.dirname(os.path.abspath("__file__"))

PDF_PATH = os.path.join(SCRIPT_DIR, "..", "data", "aqua_202005.pdf")
VECTORSTORE_PATH = os.path.join(SCRIPT_DIR, "..", "data", "faiss_index")


embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
pdf_text = extract_text_from_pdf(PDF_PATH, start_page=391, end_page=469)
texts = split_text(pdf_text)

vectorstore = create_vectorstore(
    texts, OPENAI_API_KEY
    )


In [11]:
retriever = vectorstore.as_retriever()

query = "車が水没しました"

context_docs = retriever.invoke(query)
print(f"len= {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata: {first_doc.metadata}")

print(first_doc.page_content)

len= 4
metadata: {}
390 7-1. まず初めに
水没したときは
万一、車両が水没したときは、落ち着いて次のように対処してください。
●
まずシートベルトをはずしてください。
●
ドアを開けることができる場合、ドアを開けて車外に出てください。
●
ドアを開けることができない場合、パワーウインドウスイッチでドア
ガラスを開けて窓から車外に出てください。
●
パワーウインドウスイッチでドアガラスが開けられない場合、落ち着


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
prompt = ChatPromptTemplate.from_template('''\
    以下の質問に対して、
    {context}をもとに回答してください。\n\n
    質問: {question}\n回答:
    ''')
                                        

model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4o-mini",
    temperature=0.7
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)




車が水没した場合は、以下の手順に従って対処してください。

1. **落ち着いて行動する**：まずは冷静になり、状況を確認してください。
2. **シートベルトを外す**：安全のために、まずシートベルトを外してください。
3. **ドアを開ける**：可能であれば、ドアを開けて車外に出てください。
4. **ドアが開けられない場合**：ドアが開かない場合は、パワーウインドウスイッチを使って窓を開け、そこから車外に出ることを試みてください。
5. **パワーウインドウが動かない場合**：万が一、パワーウインドウが動かない場合は、冷静に次の行動を考え、他の出口を探してください。

安全が最優先ですので、焦らずに行動してください。
